### monthly revenue mart

In [0]:
%sql
-- To track how much revenue is generated each month
create or replace table nyc_taxi.gold.monthly_revenue_mart as
select
  pickup_year,
  pickup_month,
  count(*) as total_trips,
  round(sum(fare_amount),2) AS total_fare,
  round(sum(tip_amount),2) AS total_tips,
  round(sum(total_amount),2) AS total_revenue
FROM nyc_taxi.gold.fact_trips
GROUP BY pickup_year, pickup_month;

### average speed by hour mart

In [0]:
%sql
--- To analyze trip speed trends by hour to identify traffic patterns
CREATE OR REPLACE TABLE nyc_taxi.gold.avg_speed_by_hour_mart AS
SELECT
  pickup_hour,
  ROUND(AVG(trip_speed_mph), 2) AS avg_speed_mph,
  COUNT(*) AS num_trips
FROM nyc_taxi.gold.fact_trips
GROUP BY pickup_hour
ORDER BY pickup_hour;

### Zone to Zone Trip Volume Mart

In [0]:
%sql
--- Understand high-volume routes between pickup and dropoff zones.
CREATE OR REPLACE TABLE nyc_taxi.gold.zone_to_zone_trip_volume_mart AS
SELECT
  dim_pickup.zone AS pickup_zone,
  dim_dropoff.zone AS dropoff_zone,
  COUNT(*) AS total_trips
FROM nyc_taxi.gold.fact_trips fact
JOIN nyc_taxi.gold.dim_location dim_pickup
  ON fact.pickup_location_id = dim_pickup.location_id
JOIN nyc_taxi.gold.dim_location dim_dropoff
  ON fact.dropoff_location_id = dim_dropoff.location_id
GROUP BY pickup_zone, dropoff_zone
ORDER BY total_trips DESC;

### Weekday vs Weekend Trip Patterns

In [0]:
%sql
--- To compare trip behavior across weekdays and weekends
CREATE OR REPLACE TABLE nyc_taxi.gold.weekday_vs_weekend_mart AS
SELECT
  CASE 
    WHEN pickup_dayofweek IN (1, 7) THEN 'Weekend'
    ELSE 'Weekday'
  END AS day_type,
  COUNT(*) AS trip_count,
  ROUND(AVG(trip_distance), 2) AS avg_distance,
  ROUND(AVG(total_amount), 2) AS avg_revenue
FROM nyc_taxi.gold.fact_trips
GROUP BY day_type;